# Apartat A

In [1]:
import ast

import pandas as pd
import numpy as np

from sklearn import linear_model, svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn import neighbors, tree, naive_bayes
from sklearn.metrics import f1_score
from sklearn.preprocessing import MinMaxScaler

import scipy

import matplotlib.pyplot as plt

import seaborn as sns

### APARTAT A

In [2]:
# EDA
#41 Atributs
#tipus definits a baix
#El target és un tipus int (0,1) depenen de si és llegendari o no (dos classes)
#taula de correlacions amb els valors de baix
#histogrames a baix, majoria balancejades pero l'atribut objectiu hi ha pocs de classe 1 (llegendaris)

#muy pocos legendarios, estudiamos datasets con igual numero de pokemons legendarios y no legendarios o todos juntos? pq tambien es importante que existen pocos legendarios


**name**: The English name of the Pokemon\
**japanese_name**: The Original Japanese name of the Pokemon\
**pokedex_number**: The entry number of the Pokemon in the National Pokedex\
**percentage_male**: The percentage of the species that are male. Blank if the Pokemon is genderless.\
**type1**: The Primary Type of the Pokemon\
**type2**: The Secondary Type of the Pokemon\
**classification**: The Classification of the Pokemon as described by the Sun and Moon Pokedex\
**height_m**: Height of the Pokemon in metres\
**weight_kg**: The Weight of the Pokemon in kilograms\
**capture_rate**: Capture Rate of the Pokemon\
**baseeggsteps**: The number of steps required to hatch an egg of the Pokemon\
**abilities**: A stringified list of abilities that the Pokemon is capable of having\
**experience_growth**: The Experience Growth of the Pokemon\
**base_happiness**: Base Happiness of the Pokemon\
**against_?**: Eighteen features that denote the amount of damage taken against an attack of a particular type\
**hp**: The Base HP of the Pokemon\
**attack**: The Base Attack of the Pokemon\
**defense**: The Base Defense of the Pokemon\
**sp_attack**: The Base Special Attack of the Pokemon\
**sp_defense**: The Base Special Defense of the Pokemon\
**speed**: The Base Speed of the Pokemon\
**generation**: The numbered generation which the Pokemon was first introduced\
**is_legendary**: Denotes if the Pokemon is legendary.

In [4]:
df = pd.read_csv('pokemon.csv')

In [39]:
OBJECTIVE = 'is_legendary'
FILTER_RATE = 0.1
NORMAL_LIMIT = 1e-3
NAMES = df['name']

In [5]:
pd.concat(
    [df.dtypes, df.corr()[OBJECTIVE], df.isnull().sum()],
    axis=1,
    keys=['dtypes', 'corr', 'null']
)

,dtypes,corr,null
abilities,object,NaN,0
against_bug,float64,0.027864,0
against_dark,float64,0.136315,0
against_dragon,float64,0.014844,0
against_electric,float64,-0.023151,0
against_fairy,float64,0.050165,0
against_fight,float64,-0.059132,0
against_fire,float64,-0.011073,0
against_flying,float64,-0.062214,0
against_ghost,float64,0.170746,0


In [6]:
# hist = df.hist(figsize=(16, 16))

In [7]:
df[OBJECTIVE].value_counts()

0    731
1     70
Name: is_legendary, dtype: int64

**Preprocessing**

In [8]:
# 2. Preprocessing
#No estàn normalitzades, 
#No té sentit intentar normalitzar ja que la majoria de dades que tenim són binàries
#No tenim gaires nan's excepte en el type2, no podem reomplir-les degut a que són strings
# (faltar contestar mes coses aqui...) Canvis dataset: En la columna capture_rate hi ha un valor que és de tipus object que no té sentit, hem ficat el valor de 70 al buscar el capture rate del pokemon a internet
# Tenim algunes variables categòriques com: classification, type1, type2, name i japanese_name, abilities (lista)
#

In [9]:
df.at[df['name'] == 'Minior', 'capture_rate'] = 70  # Because '30 (Meteorite)255 (Core)'
df['capture_rate'] = df['capture_rate'].astype('int64')

In [10]:
df.describe().loc[['mean', '50%', 'std']]

,against_bug,against_dark,against_dragon,against_electric,against_fairy,against_fight,against_fire,against_flying,against_ghost,against_grass,...,height_m,hp,percentage_male,pokedex_number,sp_attack,sp_defense,speed,weight_kg,generation,is_legendary
mean,0.996255,1.057116,0.968789,1.073970,1.068976,1.065543,1.135456,1.192884,0.985019,1.034020,...,1.163892,68.958801,55.155761,401.000000,71.305868,70.911361,66.334582,61.378105,3.690387,0.087391
50%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,65.000000,50.000000,401.000000,65.000000,66.000000,65.000000,27.300000,4.000000,0.000000
std,0.597248,0.438142,0.353058,0.654962,0.522167,0.717251,0.691853,0.604488,0.558256,0.788896,...,1.080326,26.576015,20.261623,231.373075,32.353826,27.942501,28.907662,109.354766,1.930420,0.282583


In [11]:
# Gaussian if (mediana y media similar y desv.std entre el 20 y 30% de la media)
desc = df.describe().loc[['mean', '50%', 'std']]

In [12]:
for col in df.select_dtypes(exclude='object'):
    stat, pvalue = scipy.stats.normaltest(df[col])
    if pvalue > NORMAL_LIMIT:
        print(col)

In [13]:
#atributs que no es poden normalitzar: generation, is_legendary, els que són objectes

In [21]:
df_2 = df.copy(deep=True)
df_2['abilities'] = df['abilities'].apply(ast.literal_eval)
ls = df_2['abilities'].explode()
df_2[ls.unique()] = 0

for idx, row in df_2.iterrows():
    for cat in row['abilities']:
        df_2.at[idx, cat] = 1
    
    # for cat in ls.unique():
    #     if(cat in df['abilities'][idx] ):
    #         df[cat][idx]=1
df_2.drop(columns='abilities', inplace=True)
df_2.shape

(801, 267)

In [43]:
df_2 = df_2.select_dtypes(exclude='object')
# df_2.drop(columns=['classfication', 'type1', 'type2', 'name', 'japanese_name'], inplace=True)
for col in ['classfication', 'type1', 'type2']:
    df_2 = pd.concat([df_2, pd.get_dummies(df[col])], axis=1)

In [30]:
corr = df_2.corr().abs()['is_legendary'].sort_values(ascending=False)
corr[corr > FILTER_RATE]

is_legendary       1.000000
base_egg_steps     0.873488
base_total         0.485440
base_happiness     0.413108
sp_attack          0.406281
                     ...   
Cavern Pokémon     0.114252
dragon             0.113667
against_psychic    0.106047
steel              0.101196
Clear Body         0.100196
Name: is_legendary, Length: 105, dtype: float64

In [33]:
df_2.dtypes

against_bug         float64
against_dark        float64
against_dragon      float64
against_electric    float64
against_fairy       float64
                     ...   
poison                uint8
psychic               uint8
rock                  uint8
steel                 uint8
water                 uint8
Length: 891, dtype: object

In [44]:
from sklearn.preprocessing import MinMaxScaler
cols = df_2.columns
scaler = MinMaxScaler()
df_3=scaler.fit_transform(df_2)
pd.DataFrame(df_3,
columns=cols)

,against_bug,against_dark,against_dragon,against_electric,against_fairy,against_fight,against_fire,against_flying,against_ghost,against_grass,...,ghost,grass,ground,ice,normal,poison,psychic,rock,steel,water
0,0.200000,0.200000,0.50,0.125,0.066667,0.125,0.466667,0.466667,0.250,0.000000,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.200000,0.200000,0.50,0.125,0.066667,0.125,0.466667,0.466667,0.250,0.000000,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.200000,0.200000,0.50,0.125,0.066667,0.125,0.466667,0.466667,0.250,0.000000,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.066667,0.200000,0.50,0.250,0.066667,0.250,0.066667,0.200000,0.250,0.066667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.066667,0.200000,0.50,0.250,0.066667,0.250,0.066667,0.200000,0.250,0.066667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
796,0.000000,0.200000,0.25,0.500,0.066667,0.250,0.466667,0.066667,0.250,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
797,0.200000,0.200000,0.25,0.125,0.066667,0.500,1.000000,0.200000,0.250,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
798,0.466667,0.066667,1.00,0.125,1.000000,0.500,0.066667,0.200000,0.125,0.066667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
799,0.466667,0.466667,0.50,0.250,0.200000,0.125,0.200000,0.200000,0.500,0.200000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c3c3f0e5-53b5-41aa-b55f-8ff8474beec2' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>